In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Define a simple fully connected neural network model for tabular data
class SimpleTabularNN(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SimpleTabularNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)  # Adjust num_classes based on your task
        )

    def forward(self, x):
        x = self.fc_layers(x)
        return x

# Define a custom dataset for your tabular data
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y

# Load your tabular data from a CSV file
csv_file_path = '/notebooks/train.csv'
data = pd.read_csv(csv_file_path)
data['row_id'] = data['row_id'].str.replace('_', '').astype(int)

# Extract features and labels
features = torch.tensor(data.drop('target', axis=1).values, dtype=torch.float32)
labels = torch.tensor(data['target'].values, dtype=torch.float32)

# Split your data into training and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create data loaders for training and validation
batch_size = 32
train_dataset = CustomDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_features, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize the model
num_features = features.shape[1]
num_classes = 1  # Adjust based on your task
model = SimpleTabularNN(num_features, num_classes)

# Define a loss function (e.g., Mean Squared Error) and optimizer (e.g., SGD)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10  # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        labels = labels.view(-1, 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# Validation loop
model.eval()
total_loss = 0.0
total_samples = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        total_samples += 1

mean_validation_loss = total_loss / total_samples
print(f"Mean Validation Loss: {mean_validation_loss:.4f}")
